In [1]:
import numpy
import scipy.sparse
from scipy.sparse.linalg import spsolve

# import quadpy

import meshio
import igl

from collections import namedtuple

In [2]:
import sys
import pathlib

sys.path.append(str(pathlib.Path(".").resolve().parents[0] / "tools" / "weights" / "L2_projection" / "build"))

import L2

In [3]:
# some quadrature rule
# scheme = quadpy.t2.get_good_scheme(48)
# scheme.show()
# scheme.integrate(lambda x: numpy.ones(x.shape[1]), [[0.0, 0.0], [1.0, 0.0], [0.0, 1.0]])
# numpy.savetxt("../tools/weights/L2_projection/quadrature/points.csv", scheme.points.T.tolist(), delimiter=",")
# numpy.savetxt("../tools/weights/L2_projection/quadrature/weights.csv", scheme.weights.tolist(), delimiter=",")
scheme_points = numpy.loadtxt("../tools/weights/L2_projection/quadrature/points.csv", delimiter=",").T
scheme_weights = numpy.loadtxt("../tools/weights/L2_projection/quadrature/weights.csv", delimiter=",")

# Python Implementation

In [4]:
def hat_phi0(x):
    return 1-x[0]-x[1]
def hat_phi1(x):
    return x[0]
def hat_phi2(x):
    return x[1]

In [5]:
Basis = namedtuple("Basis", ["n_bases", "phi", "loc_2_glob", "gmapping", "grad_gmapping"])

def build_bases(V, F):
    return [Basis(
        n_bases=3,
        phi=[hat_phi0, hat_phi1, hat_phi2], 
        loc_2_glob=f.copy(),
        gmapping = lambda x, f=f: (V[f[0]] * x[0] + V[f[1]] * x[1] + V[f[2]] * x[2]),
        grad_gmapping=numpy.linalg.norm(numpy.cross(V[f[1]] - V[f[0]], V[f[2]] - V[f[0]]))
    ) for f in F]

In [6]:
def compute_mass_mat(elements):
    rows, cols, vals = [], [], []
    
    q_pts = scheme_points.T
    q_w = scheme_weights

    # same as above but now we use phi instead of grad_phi and no division
    for el in elements:
        for i, phi_i in enumerate(el.phi):
            for j, phi_j in enumerate(el.phi):
                # ∫_{ŝ_j} ϕ̂ᵢ⋅ϕ̂ⱼ \,(s_{j, 1} - s_{j, 0})
                val = 0
                for t, w in zip(q_pts, q_w):
                    x = t[1:]
                    val += w * phi_i(x) * phi_j(x) * el.grad_gmapping

                rows.append(el.loc_2_glob[i])
                cols.append(el.loc_2_glob[j])
                vals.append(val)

    return scipy.sparse.coo_matrix((vals, (rows, cols))).tocsc()
    

In [7]:
def point_triangle_coord(p, a, b, c):
    return igl.barycentric_coordinates_tri(
        p.reshape((1, -1)).copy(), a.reshape((1, -1)).copy(), 
        b.reshape((1, -1)).copy(), c.reshape((1, -1)).copy())

In [8]:
def eval_phi_j(bases, index, x):
    el = bases[index]
    out = []
    for j in range(el.n_bases):
        out.append((el.loc_2_glob[j], el.phi[j](x)))
    return out

In [9]:
from tqdm import tqdm

def compute_mass_mat_cross(Va, Fa, a, Vb, Fb, b):
    rows, cols, vals = [], [], []
    
    q_pts = scheme_points.T
    q_w = scheme_weights

    ###########################################################################
    # Embree ray cast
    Na = igl.per_vertex_normals(Va, Fa)
    origins = numpy.empty((len(a) * 3 * len(q_pts), 3))
    rays = numpy.empty(origins.shape)
    ray_i = 0
    for el in tqdm(a):
        for _ in range(el.n_bases):
            for t in q_pts:
                origins[ray_i] = el.gmapping(t)
                rays[ray_i] = (t * Na[el.loc_2_glob]).sum(axis=0)
                ray_i += 1

    print("Computing closest points")
    ids_and_coords = L2.igl_embree_line_mesh_intersection(origins, rays, Vb, Fb)
    
    query_i = 0
    for el in tqdm(a):
        for i in range(el.n_bases):
            others = {}
            for t, w in zip(q_pts, q_w):
                bb = eval_phi_j(b, int(ids_and_coords[query_i, 0]), 
                                ids_and_coords[query_i, 1:])
                query_i += 1
                
                x = t[1:]
                for j, phi_j in bb:
                    if j not in others:
                        others[j] = 0
                    others[j] += w * phi_j * el.phi[i](x) * el.grad_gmapping
            
            for j, val in others.items():
                rows.append(el.loc_2_glob[i])
                cols.append(j)
                vals.append(val)

    return scipy.sparse.coo_matrix((vals, (rows, cols))).tocsc()

# Meshes

In [10]:
coarse_mesh = meshio.read("../meshes/sphere/sphere-coarse.msh")
V_coarse = coarse_mesh.points
F_coarse = coarse_mesh.cells[0].data.astype(int)
F_coarse = igl.boundary_facets(F_coarse)
V_coarse, F_coarse, *_ = igl.remove_unreferenced(V_coarse, F_coarse)

fine_mesh = meshio.read("../meshes/sphere/sphere-fine.obj")
V_fine = fine_mesh.points
F_fine = fine_mesh.cells[0].data.astype(int)
# E_fine = igl.boundary_facets(fine_mesh.cells[0].data.astype(int))
# V_fine, E_fine, *_ = igl.remove_unreferenced(V_fine, E_fine[:, [0, 1, 1]])

# point_triangle_coord(V_coarse[0], *V_fine[F_fine[10]])

In [11]:
U_coarse = numpy.zeros(V_coarse.shape)
U_coarse[:, 1] += numpy.cos(1 * numpy.pi * V_coarse[:, 0])

U_fine = numpy.zeros(V_fine.shape)
U_fine[:, 1] += numpy.cos(1 * numpy.pi * V_fine[:, 0])

In [12]:
coarse_coll = True
if coarse_coll:
    V_coll, V_fem = V_coarse, V_fine
    F_coll, F_fem = F_coarse, F_fine
    U_coll, U_fem = U_coarse, U_fine
else:
    V_coll, V_fem = V_fine, V_coarse
    F_coll, F_fem = F_fine, F_coarse
    U_coll, U_fem = U_fine, U_coarse

# V_fem = V_coll
# E_fem = E_coll
# U_fem = U_coll

## Test Python Implementation

In [13]:
A = V_coll
F_A = F_coll
B = V_fem
F_B = F_fem

In [14]:
A_bases = build_bases(A, F_A)
B_bases = build_bases(B, F_B)

In [15]:
MM_python = compute_mass_mat(A_bases)
assert(MM_python.shape == (A.shape[0], A.shape[0]))

In [16]:
M_python = compute_mass_mat_cross(A, F_A, A_bases, B, F_B, B_bases)
print(M_python.shape, (A.shape[0], B.shape[0]))

100%|██████████| 460/460 [00:03<00:00, 125.27it/s]


Computing closest points


100%|██████████| 460/460 [00:01<00:00, 260.30it/s]

(232, 2562) (232, 2562)


# Test C++ Implementation

In [17]:
quadrature = L2.Quadrature()
coll_bases = L2.build_bases(V_coll, F_coll)
fem_bases = L2.build_bases(V_fem, F_fem)

In [18]:
MM_cpp = L2.compute_mass_mat(V_coll.shape[0], coll_bases, quadrature)
numpy.linalg.norm(MM_python.A - MM_cpp.A)

0.11190362141661483

In [19]:
M_cpp = L2.compute_mass_mat_cross(
    V_fem, F_fem, fem_bases, V_coll, F_coll, coll_bases, quadrature)
numpy.linalg.norm(M_python.A - M_cpp.A, numpy.Inf)

TypeError: compute_mass_mat_cross(): incompatible function arguments. The following argument types are supported:
    1. (V_fem: numpy.ndarray[numpy.float64[m, n]], F_fem: numpy.ndarray[numpy.int32[m, n]], fem_bases: List[L2.Basis], V_coll: numpy.ndarray[numpy.float64[m, n]], F_coll: numpy.ndarray[numpy.int32[m, n]], coll_bases: List[L2.Basis], closest_points: Callable[[numpy.ndarray[numpy.float64[m, n]], numpy.ndarray[numpy.float64[m, n]], numpy.ndarray[numpy.float64[m, n]], numpy.ndarray[numpy.int32[m, n]]], numpy.ndarray[numpy.float64[m, n]]], quadrature: L2.Quadrature = <L2.Quadrature object at 0x10c10ccf0>) -> scipy.sparse.csc_matrix[numpy.float64]

Invoked with: array([[ 0.      , -0.5     ,  0.      ],
       [ 0.361804, -0.22361 ,  0.262863],
       [-0.138194, -0.22361 ,  0.425325],
       ...,
       [ 0.297514, -0.311916,  0.253365],
       [ 0.31453 , -0.284579,  0.264737],
       [ 0.329248, -0.257551,  0.274341]]), array([[   0,   27,   26],
       [   1,   41,   71],
   

In [ ]:
W_cpp = L2.L2_projection(V_fem,F_fem,V_coll,F_coll, False)

AttributeError: module 'L2' has no attribute 'L2_projection'

In [ ]:
MM = MM_cpp
# M = M_python
M = M_cpp

NameError: name 'M_cpp' is not defined

# Solve System

In [ ]:
def density(A):
    return (A.nnz / numpy.product(A.shape)) * 100

def print_density(A):
    print(f"{density(A):.2f}%")

def eliminate_near_zeros(A, tol=1e-12):
    A.data[numpy.abs(A.data) < tol] = 0
    A.eliminate_zeros()

def unity_error(W):
    return numpy.max(abs(W.sum(axis=1) - 1))

In [ ]:
eliminate_near_zeros(W_cpp)
print_density(W_cpp)
print(unity_error(W_cpp))

NameError: name 'W_cpp' is not defined

In [ ]:
W = spsolve(MM, M)
eliminate_near_zeros(W)
print_density(W)
print(unity_error(W))

NameError: name 'M' is not defined

In [ ]:
MM_lumped = scipy.sparse.csc_matrix(MM.shape)
MM_lumped.setdiag(MM.sum(axis=1))
W_lumped = spsolve(MM_lumped, M)
eliminate_near_zeros(W_lumped)
print(f"{numpy.linalg.norm((W - W_lumped).A)} {density(W_lumped):.2f}% {unity_error(W_lumped)}")

/opt/homebrew/lib/python3.11/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


NameError: name 'M' is not defined

In [ ]:
# print("cg_tol: error, density")
# for tol in numpy.geomspace(1e-6, 1e-3, 4):
#     W_CG = numpy.hstack([
#         scipy.sparse.linalg.cg(MM, M.A[:, i], tol=tol)[0].reshape(-1, 1)
#         for i in range(M.shape[1])])
#     W_CG = scipy.sparse.csc_matrix(W_CG)
#     eliminate_near_zeros(W_CG)
#     print(f"{tol}: {numpy.linalg.norm((W - W_CG).A)} {density(W_CG):.2f}%")
# W_CG /= W_CG.sum(axis=1)
# W_CG = scipy.sparse.csc_matrix(W_CG)
# eliminate_near_zeros(W_CG)
# print(f"{tol}: {numpy.linalg.norm((W - W_CG).A)} {density(W_CG):.2f}% {unity_error(W_CG)}")

In [ ]:
# print("drop_tol: error, density, unity error")
# for drop_tol in numpy.geomspace(1e-4, 1e-2, 3):
#     MM_LU = scipy.sparse.linalg.spilu(MM, drop_tol=drop_tol)
#     W_spilu = numpy.hstack([
#         MM_LU.solve(M.A[:, i]).reshape(-1, 1) for i in range(M.shape[1])])
#     W_spilu = scipy.sparse.csc_matrix(W_spilu)
#     eliminate_near_zeros(W_spilu)
#     print(f"{drop_tol}: {numpy.linalg.norm((W - W_spilu).A)} {density(W_spilu):.2f}% {unity_error(W_CG)}")

In [ ]:
# print_density(M)
# print_density(MM)
# print(W.shape, U_fem.shape, V_coll.shape)

# Visualization

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
colors = ["blue", "red", "green", "orange", "cyan"]

count = 0

def MeshTrace(V, F, mode="lines", name="", opacity=1):
    E = igl.edges(F)
    segments_x = []
    segments_y = []
    segments_z = []
    for e in E:
        segments_x.extend(V[e, 0].tolist() + [None])
        segments_y.extend(V[e, 1].tolist() + [None])
        segments_z.extend(V[e, 2].tolist() + [None])
    global count
    count += 1
    return [
        go.Scatter3d(x=segments_x, y=segments_y, z=segments_z, mode=mode, name=name, line={"color": "black"}),
        go.Mesh3d(x=V[:, 0], y=V[:, 1], z=V[:, 2], i=F[:, 0], j=F[:, 1], k=F[:, 2], name=name, flatshading=True, color=colors[count-1], opacity=opacity)
    ]

In [ ]:
count = 0
X_coll = V_coll + W_cpp @ U_fem
X_coll_lumped = V_coll + W_cpp @ U_fem
X_fem = V_fem + U_fem

fig = make_subplots(rows=2, cols=2,
    specs=[[{'type': 'surface'}, {'type': 'surface'}],
           [{'type': 'surface'}, {'type': 'surface'}]])

for trace in MeshTrace(V_fem, F_fem, name="V_fem"):
    fig.add_trace(trace, row=1, col=1)
for trace in MeshTrace(V_coll, F_coll, name="V_coll"):
    fig.add_trace(trace, row=1, col=2)
for trace in MeshTrace(X_fem, F_fem, name="Deformed V_fem"):
    fig.add_trace(trace, row=2, col=1)
for trace in MeshTrace(X_coll, F_coll, name="Deformed V_coll"):
    fig.add_trace(trace, row=2, col=2)
for trace in MeshTrace(X_coll_lumped, F_coll, name="Deformed (Lumped) V_coll", opacity=0.5):
    fig.add_trace(trace, row=2, col=2)

fig.update_layout(width=1500, height=1500, title="Displace V_fem", showlegend=True)
fig.update_yaxes(scaleanchor="x", scaleratio=1)

NameError: name 'W_cpp' is not defined

In [ ]:
# X_coll = V_coll + U_coll
# X_fem = V_fem + W.T @ U_coll
# X_fem_lumped = V_fem + W_lumped.T @ U_coll

# fig = make_subplots(rows=2, cols=2)

# fig.add_trace(MeshTrace(V_coll, E_coll, name="V_coll"), row=1, col=1),
# fig.add_trace(MeshTrace(V_fem, E_fem, name="V_fem"), row=1, col=2),
# fig.add_trace(MeshTrace(X_coll, E_coll, name="Deformed V_coll"), row=2, col=1),
# fig.add_trace(MeshTrace(X_fem, E_fem, name="Deformed V_fem"), row=2, col=2),
# fig.add_trace(MeshTrace(X_fem_lumped, E_fem, name="Deformed (lumped) V_fem"), row=2, col=2),

# fig.update_layout(width=800, height=800, title="Displace V_fem")

# None